In [1]:
import statsmodels.api as sm #for OLS regression
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt #for heatmap plotting
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns #for heatmap plotting
from statsmodels.iolib.summary2 import summary_col #to summarize multiple regressions in one table
from datetime import date
from dateutil.relativedelta import relativedelta
import math
from scipy import stats

In [3]:
ff5 = pd.read_csv('C:/Users/yzou/Desktop/F-F_Research_Data_5_Factors_2x3_daily.csv')

In [4]:
ff5.rename(columns = {'Unnamed: 0': 'YYMM'}, inplace = True)

In [5]:
ff5['YYMM'] = pd.to_datetime(ff5['YYMM'],yearfirst=True, format='%Y%m%d')

In [6]:
ff5.index = ff5['YYMM']

In [7]:
ff5 = ff5.drop('RF', axis = 1)

In [10]:
ff5 = ff5.drop('YYMM', axis=1)

In [13]:
models_ff = []
for i in range (5):
    x = ff5.drop(ff5.columns[i], axis=1)
    x = sm.add_constant(x)
    y = ff5.iloc[:, i]
    reg = sm.OLS(y, x)   
    models_ff.append(reg.fit())


In [19]:
print(models_ff[4].summary())

                            OLS Regression Results                            
Dep. Variable:                    CMA   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     2184.
Date:                Wed, 10 Jul 2019   Prob (F-statistic):               0.00
Time:                        21:11:38   Log-Likelihood:                -2372.1
No. Observations:               14075   AIC:                             4754.
Df Residuals:                   14070   BIC:                             4792.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0100      0.002      4.138      0.0

In [26]:
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}
           
results_table = summary_col(results= models_ff,
                            stars = True,
                            model_names=ff5.columns.tolist(),
                            float_format ="%.2f",
                            info_dict = info_dict,
                            regressor_order=['const', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
                           )

results_table.add_title('Regressions among 5 Factors - Daily Return')

print(results_table)

                   Table - OLS Regressions
                  Mkt-RF    SMB      HML      RMW      CMA   
-------------------------------------------------------------
const            0.05***  0.02***  0.01**   0.02***  0.01*** 
                 (0.01)   (0.00)   (0.00)   (0.00)   (0.00)  
Mkt-RF                    -0.10*** -0.01**  -0.10*** -0.10***
                          (0.00)   (0.00)   (0.00)   (0.00)  
SMB              -0.33***          0.03***  -0.22*** -0.03***
                 (0.01)            (0.01)   (0.01)   (0.00)  
HML              -0.04**  0.04***           -0.08*** 0.37*** 
                 (0.02)   (0.01)            (0.01)   (0.00)  
RMW              -0.66*** -0.46*** -0.12***          0.02*** 
                 (0.02)   (0.01)   (0.01)            (0.01)  
CMA              -0.93*** -0.07*** 0.77***  0.03***          
                 (0.02)   (0.01)   (0.01)   (0.01)           
R-squared        0.21     0.12     0.32     0.16     0.38    
No. observations 14075    1

In [28]:
ff5_mm = pd.read_csv('C:/Users/yzou/Desktop/F-F_Research_Data_5_Factors_2x3_monthly.csv')

In [29]:
ff5_mm.rename(columns = {'Unnamed: 0': 'YYMM'}, inplace = True)

In [31]:
ff5_mm['YYMM'] = pd.to_datetime(ff5_mm['YYMM'],yearfirst=True, format='%Y%m')

In [32]:
ff5_mm.index = ff5_mm['YYMM']

In [33]:
ff5_mm = ff5_mm.drop('RF', axis = 1)
ff5_mm = ff5_mm.drop('YYMM', axis=1)

In [35]:
models_ff_mm = []
for i in range (5):
    x = ff5_mm.drop(ff5_mm.columns[i], axis=1)
    x = sm.add_constant(x)
    y = ff5_mm.iloc[:, i]
    reg = sm.OLS(y, x)   
    models_ff_mm.append(reg.fit())

In [36]:
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}
           
results_table = summary_col(results= models_ff_mm,
                            stars = True,
                            model_names=ff5_mm.columns.tolist(),
                            float_format ="%.2f",
                            info_dict = info_dict,
                            regressor_order=['const', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
                           )

results_table.add_title('Regressions among 5 Factors - Monthly Return')

print(results_table)

        Regressions among 5 Factors - Monthly Return
                  Mkt-RF    SMB      HML     RMW      CMA   
------------------------------------------------------------
const            0.79***  0.30***  -0.02   0.40***  0.23*** 
                 (0.15)   (0.11)   (0.08)  (0.08)   (0.05)  
Mkt-RF                    0.13***  0.02    -0.10*** -0.11***
                          (0.03)   (0.02)  (0.02)   (0.01)  
SMB              0.25***           0.03    -0.22*** -0.03   
                 (0.05)            (0.03)  (0.03)   (0.02)  
HML              0.09     0.06             0.14***  0.45*** 
                 (0.07)   (0.05)           (0.04)   (0.02)  
RMW              -0.38*** -0.43*** 0.14***          -0.14***
                 (0.07)   (0.05)   (0.04)           (0.03)  
CMA              -0.91*** -0.13    1.00*** -0.29***         
                 (0.10)   (0.08)   (0.04)  (0.06)           
R-squared        0.24     0.17     0.49    0.17     0.55    
No. observations 671      671   